# Welcome to Modal notebooks!

Write Python code and collaborate in real time. Your code runs in Modal's
**serverless cloud**, and anyone in the same workspace can join.

This notebook comes with some common Python libraries installed. Run
cells with `Shift+Enter`.

# Notebook: Sync a Local Folder with Dropbox

This notebook provides a Python script to perform a one-way synchronization from a local folder on your computer to a folder in your Dropbox account.

**Features:**
- **Uploads new files:** Any file in the local folder that isn't in the Dropbox folder will be uploaded.
- **Updates modified files:** If a file exists in both places, it will be re-uploaded if the local version has a more recent modification time.
- **(Optional) Sync Deletions:** If enabled, any file in the Dropbox folder that is *not* in the local folder will be deleted from Dropbox. **Use with caution!**
- **Handles subdirectories:** The script recursively syncs all folders and subfolders.

## Step 1: Install Required Libraries

First, we need to install the official Dropbox Python SDK and `tqdm` for a nice progress bar during uploads.

In [1]:
!pip install dropbox tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.1/572.1 kB 66.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


## Step 2: Get Your Dropbox API Access Token

To allow this script to access your Dropbox, you need to create a Dropbox App and generate an access token.

**Instructions:**

1.  **Go to the Dropbox App Console:** [https://www.dropbox.com/developers/apps](https://www.dropbox.com/developers/apps)
2.  Click **"Create app"**.
3.  Choose **"Scoped access"**.
4.  Select the type of access you need: **"App folder"** (recommended, more secure as it can only access its own folder) or **"Full Dropbox"**.
5.  Give your app a unique name (e.g., `MyPythonSyncScript`).
6.  Once the app is created, go to the **"Permissions"** tab.
7.  Check the following permissions:
    *   `files.metadata.read`
    *   `files.metadata.write`
    *   `files.content.read`
    *   `files.content.write`
8.  Click **"Submit"** to save the permissions.
9.  Go to the **"Settings"** tab.
10. Find the **"Generated access token"** section and click **"Generate"**.
11. Copy the generated token. This is what you'll use in the next step.

> **⚠️ Security Warning:**
> Treat your access token like a password. Do not share it or commit it to public code repositories like GitHub.

## Step 3: Configuration

Update the variables in this cell with your access token and the folder paths you want to sync.

In [2]:
import os
import dropbox
from dropbox.exceptions import AuthError

# --- ⚙️ START OF CONFIGURATION ⚙️ ---

# 1. PASTE YOUR DROPBOX ACCESS TOKEN HERE
DROPBOX_ACCESS_TOKEN = "sl.u.AF40gFKWO-Os6rU4_Q2lfwevhfcDZ7usRJ4PkmmbElOV4yNIwYiqQndmjHcvfC90_NxYSIV-yJ5-UiXRf6nXbu2geJD1gq7sMiB6ZQrS9g_AZX6TyQw7LT1giwNuvpNg9NNqZX4ofsPTDnxctsLhuXzMEnmGQehVaVIECqWlnyYv90knqiKOA98dX8RVQls9jlE8YvdERDoGeICt5EqTsLMoZs-sz43Bjl0kIehmkQwpX7rk83KrJ0iD8K6ixgF01ENAz8ZBQ90shpJwGMowj7tUUouC740LVMMPBMUH7tSaUgHAN6Gf2Orw2NLkSAHYV8pQiX7xmL83qpPCd4fQ7g0vNNI4PQMyk4fRP-2PPMes72idX5T4LaiPPq6r5-BQYimmYbJqkE4bdAOS9vQFoJM3j8J95JB2MWNuHucWVVGFzgFDiS9A6PK8PL0kUkjmMnZBVJPK821lXo9bKeM4tBh__5MWOLB6d7MjeaigPRsyP3NFgkG52By125YVbc34g_ict2LL2RFI4dsGu9ze9X2enUh798dMRgRHHA6iwOaM_j6zROlG7H-ykou3XE4D7PHUmCnW1iFZY7wd6ras_3bsUDGKLEYJRcnrlUsfod5pe6Myk0tEkI_aDw6Cj5-8T854GlCGBMEcusPqfVCO4DUWkjcgi-ZlmJhDHymOxd_v4QTKBof2D0lzV-tlTuOXuc1TkFE543X3PhvtGCd2f4OGDjUFXmRGgkwNS8Dwsz5hCwh60k7LW-O1Wm1ezHjCYFmQM6Qa_r6v5rrmnVQoEAkfqdJOC7AqgnuC3ukKLsSFzj_169340vpOdWYZ8epwVxtntv2kLjy1Aw-VN_Fs6NSVw4X4zTwE_wZ7J1f-ncE-9C2bbu74P0bb04prqRTRMQfIwdH5ntjS12EpgogrrACW52Db7vItXTztrI-H21Ndzvom6-3ooK1Q5E5Nv3gGwufMqx3yIthWFa0-TjAFDHaZNFjuWYU6cPn6U1jz_46m78ENDSYdJrH2fTEHwQhLabbVSNgXSOAJrQ_9rJ053x3usVxe6rUgoJQmZ3cj6tVq2Np8JgBkI6Tr7GWYFf-aliSx1VuBwdRdWDc_AlIx2nDGnFBM3EPBD4ymNYVDv0HxPDMtK_mcNjWOP3rZqHdXS821rWHezYQJJc0zK-j6dh30-A7Rrey4hpFRhE4CAiRbYm9LG5DhS-5BsEkFzoZDNKoz0bLN0eWMNTMU_TJBYqIx9Rzg5MxDL3zlkx2mu51MOA6V_bolnmuaqJtoP6PxwkeCpiD2zxoWZP_qk9yanVDc"

# 2. DEFINE THE LOCAL FOLDER TO SYNC
#    - It can be an absolute path (e.g., "C:/Users/You/Documents/Photos")
#    - Or a relative path from this notebook (e.g., "./my_local_folder")
LOCAL_FOLDER = "/mnt/my-volume/ComfyUI/models"

# 3. DEFINE THE DROPBOX FOLDER
#    - This path is relative to your Dropbox root or your App's folder.
#    - It MUST start with a forward slash '/'.
#    - E.g., "/Apps/MyPythonSyncScript/Backup"
DROPBOX_FOLDER = "/test/models"

# 4. (OPTIONAL) SYNC DELETIONS
#    - If True, files in Dropbox but NOT in the local folder will be deleted.
#    - Set to False to only upload/update files.
#    - ⚠️ THIS IS A DESTRUCTIVE OPERATION. BE CAREFUL.
SYNC_DELETIONS = False

# --- END OF CONFIGURATION ---


# --- Create a dummy local folder and files for demonstration ---
# print(f"Setting up a dummy local folder at: {LOCAL_FOLDER}")
# os.makedirs(LOCAL_FOLDER, exist_ok=True)
# os.makedirs(os.path.join(LOCAL_FOLDER, "subdir"), exist_ok=True)

# with open(os.path.join(LOCAL_FOLDER, "hello.txt"), "w") as f:
#     f.write("This is the first version of the file.")
# with open(os.path.join(LOCAL_FOLDER, "subdir", "report.csv"), "w") as f:
#     f.write("id,value\n1,100\n2,200")
# print("Dummy files created.")

## Step 4: Helper Functions and Dropbox Client Initialization

These functions will handle the details of listing files, uploading, and connecting to Dropbox.

In [3]:
import os
import time
import dropbox
from dropbox.exceptions import AuthError, ApiError
from dropbox.files import WriteMode, CommitInfo
from datetime import datetime, timezone
from tqdm import tqdm

# --- ⚙️ CONSTANTS ⚙️ ---
# Dropbox API limit for a single upload call is 150MB. We'll use 145MB as a safe threshold.
LARGE_FILE_THRESHOLD = 145 * 1024 * 1024 
# Chunk size for large file uploads. 8MB is a reasonable choice.
CHUNK_SIZE = 8 * 1024 * 1024

def connect_to_dropbox(token):
    """Initializes and returns a Dropbox client instance, testing the connection."""
    try:
        dbx = dropbox.Dropbox(token)
        dbx.users_get_current_account()
        print("✅ Successfully connected to Dropbox.")
        return dbx
    except AuthError:
        print("❌ Authentication Error: Invalid Dropbox access token.")
        return None

def get_local_files(local_path):
    """
    Recursively lists all files in the local directory.
    Returns a dictionary of {relative_path: (full_path, mtime)}.
    """
    local_files = {}
    for root, _, files in os.walk(local_path):
        for filename in files:
            full_path = os.path.join(root, filename)
            relative_path = os.path.relpath(full_path, local_path).replace(os.path.sep, '/')
            mtime = os.path.getmtime(full_path)
            local_files[relative_path] = (full_path, mtime)
    return local_files

def get_dropbox_files(dbx, dropbox_path):
    """
    Recursively lists all files in the Dropbox directory.
    Returns a dictionary of {relative_path: metadata}.
    """
    dropbox_files = {}
    try:
        result = dbx.files_list_folder(dropbox_path, recursive=True)
        while True:
            for entry in result.entries:
                if isinstance(entry, dropbox.files.FileMetadata):
                    relative_path = os.path.relpath(entry.path_display, dropbox_path).replace(os.path.sep, '/')
                    dropbox_files[relative_path] = entry
            if not result.has_more:
                break
            result = dbx.files_list_folder_continue(result.cursor)
    except ApiError as err:
        if err.error.get_path().is_not_found():
            print(f"Dropbox folder '{dropbox_path}' not found. It will be treated as empty.")
        else:
            raise
    return dropbox_files

def upload_file(dbx, local_file_path, dropbox_file_path, mtime):
    """
    Uploads a file to Dropbox, handling large files with a robust upload session.
    """
    file_size = os.path.getsize(local_file_path)
    mtime_dt = datetime.fromtimestamp(mtime, timezone.utc)
    
    with open(local_file_path, 'rb') as f:
        # No progress bar here, we'll use the main one in the sync loop.
        if file_size <= LARGE_FILE_THRESHOLD:
            # --- Standard upload for smaller files ---
            dbx.files_upload(
                f.read(),
                dropbox_file_path,
                mode=WriteMode('overwrite'),
                client_modified=mtime_dt,
                mute=True
            )
        else:
            # --- Robust upload session for larger files ---
            
            # 1. Start the session with the first chunk
            upload_session_start_result = dbx.files_upload_session_start(f.read(CHUNK_SIZE))
            
            cursor = dropbox.files.UploadSessionCursor(
                session_id=upload_session_start_result.session_id,
                offset=f.tell()
            )
            commit = CommitInfo(path=dropbox_file_path, mode=WriteMode('overwrite'), client_modified=mtime_dt, mute=True)
            
            # 2. Loop over the remaining chunks
            while f.tell() < file_size:
                # Check if the remaining bytes are less than the next chunk size.
                if (file_size - f.tell()) <= CHUNK_SIZE:
                    # If so, this is the last chunk. Finish the session.
                    chunk = f.read(CHUNK_SIZE) # Read the final chunk
                    dbx.files_upload_session_finish(chunk, cursor, commit)
                    break # Exit the loop, we are done
                else:
                    # Otherwise, append the next chunk.
                    chunk = f.read(CHUNK_SIZE)
                    dbx.files_upload_session_append_v2(chunk, cursor)
                    cursor.offset = f.tell() # Update the offset for the next iteration

    # Return the path for the main progress bar
    return os.path.basename(local_file_path)

# --- Initialize Dropbox Client ---
# Make sure DROPBOX_ACCESS_TOKEN is defined from the previous cell
dbx = connect_to_dropbox(DROPBOX_ACCESS_TOKEN)

✅ Successfully connected to Dropbox.


## Step 5: Run the Synchronization

This is the main logic cell. It compares the local and remote file lists and performs the necessary actions (upload, update, or delete).

In [ ]:
import concurrent.futures

# --- ⚙️ PARALLELISM CONFIGURATION ⚙️ ---
# Number of parallel threads to use for uploading/deleting.
# A good starting point is between 4 and 10. Too many can lead to rate limiting.
MAX_WORKERS = 10

if dbx:
    print("\n--- Starting Synchronization ---")
    
    # 1. Get file lists (this part is still serial)
    print("Listing local files...")
    local_files = get_local_files(LOCAL_FOLDER)
    print(f"Found {len(local_files)} files locally.")
    
    print("Listing remote files from Dropbox...")
    dropbox_files = get_dropbox_files(dbx, DROPBOX_FOLDER)
    print(f"Found {len(dropbox_files)} files on Dropbox.")
    
    # 2. Determine tasks to be performed
    files_to_upload = []
    files_to_delete = []

    print("\n--- Comparing file lists to determine operations ---")
    # Check for files to upload or update
    for relative_path, (full_path, mtime) in local_files.items():
        if relative_path not in dropbox_files:
            files_to_upload.append((full_path, relative_path, mtime))
        else:
            dropbox_mtime = dropbox_files[relative_path].client_modified.replace(tzinfo=timezone.utc)
            local_mtime_utc = datetime.fromtimestamp(mtime, timezone.utc)
            if (local_mtime_utc - dropbox_mtime).total_seconds() > 2:
                files_to_upload.append((full_path, relative_path, mtime))

    # Check for files to delete (if enabled)
    if SYNC_DELETIONS:
        remote_only_files = set(dropbox_files.keys()) - set(local_files.keys())
        for relative_path in remote_only_files:
            files_to_delete.append(relative_path)
            
    print(f"Found {len(files_to_upload)} files to upload/update.")
    print(f"Found {len(files_to_delete)} files to delete.")

    # 3. Execute tasks in parallel
    all_tasks = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Submit upload tasks
        for full_path, relative_path, mtime in files_to_upload:
            dropbox_path = f"{DROPBOX_FOLDER}/{relative_path}"
            task = executor.submit(upload_file, dbx, full_path, dropbox_path, mtime)
            all_tasks.append(task)
            
        # Submit delete tasks
        for relative_path in files_to_delete:
            dropbox_path = f"{DROPBOX_FOLDER}/{relative_path}"
            task = executor.submit(delete_file, dbx, dropbox_path)
            all_tasks.append(task)
            
        # --- Process results with a progress bar ---
        if all_tasks:
            print(f"\n--- Executing {len(all_tasks)} operations with {MAX_WORKERS} workers ---")
            for future in tqdm(concurrent.futures.as_completed(all_tasks), total=len(all_tasks), desc="Syncing files"):
                try:
                    result = future.result()
                    # You could add more detailed logging here if needed, e.g., print(f"Completed: {result}")
                except Exception as e:
                    print(f"❗️ A task failed with an error: {e}")
        else:
            print("\n--- No operations to perform. Folders are in sync. ---")

    print("\n\n--- Synchronization Complete ---")

else:
    print("\nCould not run sync due to connection failure.")


--- Starting Synchronization ---
Listing local files...
Found 1095 files locally.
Listing remote files from Dropbox...
Dropbox folder '/test/models' not found. It will be treated as empty.
Found 0 files on Dropbox.

--- Comparing file lists to determine operations ---
Found 1095 files to upload/update.
Found 0 files to delete.

--- Executing 1095 operations with 10 workers ---


Syncing files:  73%|███████████████████████████████████████████▏               | 802/1095 [28:33<04:47,  1.02it/s]